In [56]:
import numpy as np
import random
from tqdm.auto import tqdm
from dataclasses import dataclass
import copy
from tree_node import TreeNode
from tree import Tree
from setup import read_problem 
from generators import *

In [57]:
x,y,PROBLEM_SIZE = read_problem(0)

In [58]:
BINARY_OPERATORS = {
    "+": np.add,
    "-": np.subtract,
    "*": np.multiply,
    "/": np.divide,
    # "^": np.pow
}

#https://numpy.org/doc/2.1/reference/routines.math.html
UNARY_OPERATORS = {
        "": lambda x: x,  
        "sin": np.sin,
        "cos": np.cos,
        "tan":np.tan,
        "log": np.log,
        "arccos": np.arccos,
        "arcsin":np.arcsin,
        "arctan":np.arctan,
        "sqrt":np.sqrt,
        "cbrt":np.cbrt,
        "abs":np.abs,
        "reciprocal":np.reciprocal,
        # "exp": np.exp
    }

VARIABLES_MAP = {f"X_{i}": x[i] for i in range(PROBLEM_SIZE)}    # {'X_0': [1, 2, 3], 'X_1': [4, 5, 6], 'X_2': [7, 8, 9]}
#LEAVES = [i for i in range(-MAX_COEFFICIENT, MAX_COEFFICIENT)] + list(VARIABLES_MAP.keys())


In [59]:
def mse(y_computed: np.ndarray, y_expected: np.ndarray):
    # 100*np.square(y_train-d3584.f(x_train)).sum()/len(y_train):g}")
    return 100 * np.square(y_expected - y_computed).sum() / len(y_expected)

def mse2(y_computed: np.ndarray, y_expected: np.ndarray):
    return np.square(y_expected - y_computed).sum() / len(y_expected)


In [60]:
coefficient_ranges = get_coefficient_ranges(x,y)

constant_ranges = get_constant_ranges(y)

In [61]:
def are_compatible(operator, value, base=0):
    match operator:
        case "/":
            if not isinstance(value,np.ndarray):
                return False if value == 0 else True
            else : # check if all values are non-zero
                return False if (0 in value) else True
            
        case "^":
            if not isinstance(value, np.ndarray) and not isinstance(base, np.ndarray): # (non array) ^ (non array)
                return False if (base < 0 and not int(value) == value) or (base == 0 and value < 0) else True
            elif not isinstance(value, np.ndarray) and isinstance(base, np.ndarray): # array ^ (non array)
                return False if (not int(value) == value and np.any(base < 0)) or (value < 0 and np.any(base == 0)) else True
            elif isinstance(value, np.ndarray) and not isinstance(base, np.ndarray): # (non array) ^ array
                return False if (base < 0 and any((not int(i) == i) for i in value)) or (base == 0 and np.any(value < 0)) else True
            else : # array ^ array
                for i in range(len(value)):                    
                    if (base[i] < 0 and not int(value[i]) == value[i]) or (base[i] == 0 and value[i] < 0):
                        return False 
                return True
            
        case "log" :
            if not isinstance(value,np.ndarray):
                return False if value <= 0 else True
            else : # check if all values are non-negative
                return False if (np.any(value <= 0)) else True
        
        case "arccos" :
            if not isinstance(value,np.ndarray):
                return False if value < -1 or value > 1 else True
            else : # check if all values are between -1 and 1
                return False if (np.any(value < -1 ) or np.any(value > 1) ) else True
            
        case "arcsin" :
            if not isinstance(value,np.ndarray):
                return False if value < -1 or value > 1 else True
            else : # check if all values are between -1 and 1
                return False if (np.any(value < -1 )or np.any(value > 1) )else True
        
        case "sqrt" :
            if not isinstance(value,np.ndarray):
                return False if value < 0 else True
            else : # check if all values are non-negative
                return False if (np.any(value < 0) )else True

        case "reciprocal" :
            if not isinstance(value,np.ndarray):
                return False if value == 0 else True
            else:
                return False if (np.any(value == 0)) else True
            
        case "tan" :
            if not isinstance(value,np.ndarray):
                k = (value - np.pi / 2) / np.pi
                return False if k.is_integer() else True
            else:
                for i in range(len(value)):
                    k = (value[i] - np.pi / 2) / np.pi
                    if k.is_integer():
                        return False
                return True

        case _:
            return True


In [ ]:
# def similarity(operator, x_i, y):
#     return 1 / (1+ mse2(operator(x_i), y))

def similarity(operator, x_i, y):
    # Add normalization to make comparison fairer across different scales
    y_pred = operator(x_i)
    y_norm = (y - y.mean()) / (y.std() + 1e-8)
    y_pred_norm = (y_pred - y_pred.mean()) / (y_pred.std() + 1e-8)
    return 1 / (1 + mse2(y_pred_norm, y_norm))

# def compute_weights_corr():
#     weights = {op: 0 for op in UNARY_OPERATORS.keys()}
    
#     for op in weights:
#         # max_sim = 0
#         max_correlation = 0
#         for i in range(np.shape(x)[0]):
#             if (are_compatible(op, x[i])):
#                 # sim = similarity(UNARY_OPERATORS[op], x[i], y)
#                 # max_sim = max(max_sim, sim)
#                 transformed = UNARY_OPERATORS[op](x[i])
#                 correlation = abs(stats.pearsonr(transformed, y)[0])
#                 max_correlation = max(max_correlation, correlation)
#         # weights[op] = max_sim
#         weights[op] = max_correlation
    
#       # Normalize weights
#     total = sum(weights.values())
#     if total > 0:
#         weights = {k: v/total for k, v in weights.items()}
#     else:
#         # If all correlations failed, use uniform weights
#         weights = {k: 1.0/len(UNARY_OPERATORS) for k in UNARY_OPERATORS.keys()}
        
#     return weights

def compute_weights_sim(operators):
    weights = {op: 0 for op in operators}
    
    for op in weights:
        max_sim = 0
        for i in range(np.shape(x)[0]):
            if (are_compatible(op, x[i])):
                sim = similarity(UNARY_OPERATORS[op], x[i], y)
                max_sim = max(max_sim, sim)
               
        weights[op] = max_sim
    
      # Normalize weights
    total = sum(weights.values())
    if total > 0:
        weights = {k: v/total for k, v in weights.items()}
    else:
        # If all correlations failed, use uniform weights
        weights = {k: 1.0/len(UNARY_OPERATORS) for k in UNARY_OPERATORS.keys()}
        
    return weights

w =  compute_weights_sim(UNARY_OPERATORS.keys())
print(w)
print(sum(w.values()))



In [63]:
# # weights
# UNARY_OPERATORS_WEIGHTS = compute_weights_sim()
# BINARY_OPERATORS_WEIGHTS = [[1/len(BINARY_OPERATORS.keys()) for _ in range(len(BINARY_OPERATORS.keys()))]]
# # UNARY_OPERATORS_WEIGHTS

## Tree helper functions

In [64]:
def random_initial_tree(depth, maxdepth, variables, binary_operators, unary_operators):
    if depth == maxdepth:  # Add a variable until they are all chosen, if yes add a number
        if variables:
            var = random.choice(variables)
            leaf = TreeNode(var)
            leaf.coefficient = compute_coefficient(var, VARIABLES_MAP, coefficient_ranges)
            variables.remove(var)
        else:
            # leaf = TreeNode(compute_coefficient())  # TODO change to compute constant
            leaf = TreeNode(generate_safe_constant(y))
            leaf.coefficient = 1
        leaf.depth = 1
        return leaf
    
    elif depth == maxdepth - 1: # Add a unary operator
        node = TreeNode(None)
        node.right = random_initial_tree(depth + 1, maxdepth, variables, binary_operators, unary_operators)
        node.left = None
    
        available_unary = [op for op in unary_operators if are_compatible(op, np.multiply(VARIABLES_MAP[node.right.value], node.right.coefficient) if node.right.value in VARIABLES_MAP else node.right.value)]
        available_weights = list(compute_weights_sim(available_unary).values())
        node.value = np.random.choice(available_unary, p=available_weights) # If a choice of a variant unary operator was made, choose a random variant from all the possible ones
        node.update_depth()
        return node
    
    else: # Add a binary operator
        node = TreeNode(None)
        node.left = random_initial_tree(depth + 1, maxdepth, variables, binary_operators, unary_operators)
        node.right = random_initial_tree(depth + 1, maxdepth, variables, binary_operators, unary_operators)
        available_binary = [op for op in binary_operators if are_compatible(op, node.right.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS), node.left.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS))]
        node.value = random.choice(available_binary) # Choose a random binary operator from all the possible ones
        node.update_depth()
        return node
    
def get_random_leaf():
    if random.choice([0, 1]): # TODO could choose larger subtree
        random_leaf = TreeNode(random.choice(list(VARIABLES_MAP.keys())))
        random_leaf.coefficient = compute_coefficient(random_leaf.value, VARIABLES_MAP, coefficient_ranges)
    else:
        random_leaf = TreeNode(generate_safe_constant(y))
        random_leaf.coefficient = 1
        # random_leaf.depth = 1
    return random_leaf
    

# def validate_after_replacement(root:TreeNode, replaced_node: Tree, unary_operators: list, binary_operators:list):
def validate_after_replacement(root:TreeNode, replaced_node: Tree):
    """
    Validate the tree after replacing a subtree.

    Args:
        root (TreeNode): The root of the tree.
        replaced_node (TreeNode): The node that was replaced.
        unary_operators (list): List of unary operators.
        binary_operators (list): List of binary operators.

    Returns:
        bool: True if the tree is valid, False otherwise.
    """
    
    def is_valid_node(parent: TreeNode) -> bool:
        right_val =  parent.right.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS)
        if parent.value in UNARY_OPERATORS.keys():
            return are_compatible(parent.value, right_val)
        
        elif parent.value in BINARY_OPERATORS.keys():
            left_val = parent.left.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS)
            return are_compatible(parent.value, right_val, left_val)
        return True

    current = replaced_node
    while current:
        parent = root.get_parent(root, current)
        if not parent:
            break

        if not is_valid_node(parent):
            return False
        current = parent  # Traverse up to the root

    return True
    
def swap_subtrees(source_tree, target_tree):
    """
    Try to swap subtrees from source_tree to target_tree.

    Args:
        source_tree (Tree): The source tree.
        target_tree (Tree): The target tree.

    Returns:
        bool: True if a swap was successful, False otherwise.
    """


    # nodoA = random.choice(lista di nodi di A)
    source_nodes = source_tree.get_nodes()

    while source_nodes:
        source_node = random.choice(source_nodes)
        if try_swap(source_node, target_tree):
            return True
        source_nodes.remove(source_node)
    return False

def try_swap(source_node: TreeNode, target_tree: Tree, filter_leaves_parents=False):
    unary_operators = list(UNARY_OPERATORS.keys())
    binary_operators = list(BINARY_OPERATORS.keys())

    target_nodes = target_tree.get_non_leaves_nodes()
    if filter_leaves_parents:
        # Consider only nodes whose children are not leaves to avoid adding subtree to leaf in mutation
        target_nodes = [node for node in target_nodes if node.right.right or node.right.left]
    
    while target_nodes:
        # target_node -> parent del nodo in target_tree che verrà sostituito con source_tree
        target_node = random.choice(target_nodes)

        if (target_node.value in unary_operators and are_compatible(target_node.value, source_node.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS))):
            tmp = target_node.right
            target_node.right = source_node
            if validate_after_replacement(target_tree.root, target_node):
                target_tree.update_depths_to_root(target_node)
                return True
            target_node.right = tmp

        elif (target_node.value in binary_operators):
            choice = random.choice(["right", "left"])
            if choice == "right" and are_compatible(target_node.value, source_node.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS), target_node.left.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS)):
                tmp = target_node.right
                target_node.right = source_node
                if validate_after_replacement(target_tree.root, target_node):
                    target_tree.update_depths_to_root(target_node)
                    return True
                target_node.right = tmp
            elif choice == "left" and are_compatible(target_node.value, target_node.right.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS), source_node.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS)):
                tmp = target_node.left
                target_node.left = source_node
                if validate_after_replacement(target_tree.root, target_node):
                    target_tree.update_depths_to_root(target_node)
                    return True
                target_node.left = tmp
        target_nodes.remove(target_node)
    return False

## Steps
- We treat a possible solution as a tree. The tree has attribute root, which is the root of the tree of class TreeNode and max_depth.
- Generate random tree
    - we need each variable at least once 
    - each variable has exactly one coefficient chosen as a random float number in the range [?, ?]
    - each variable has exactly one unary operator
    - unary operator is chosen as: 50% chance of "" (i.e. no change to the variable), 50% chance of choosing among all other unary operators
        - check if the unary operator is appliable to the variable ->
            ```
            leaves_map = {}
            for e in leaves:
                available_unary_operators = [op for op in list(UNARY_OPERATORS.keys()) if op.is_applicable(e)]
                chosen_unary_operator = 50% chance of "" (i.e. no change to the variable), 50% chance of choosing among [available_unary_operators]
                leaves_map[e] = [chosen_unary_operator]
            # leaves = [-2, 3]
            # leaves_map = {-2: square, 3: log}
            for e in leaves:
                node = leaves_map[e]
                node.left = null
                node.right = e
                # insert node to tree
            ```
    - number of leaves = nearest power of two greater than keys.length()
    - number of actual leaves = [number of leaves] * 2
    - number of coefficients = [number of leaves] - keys.length()
    - number of binrary operators = total number of nodes in  tree with [number of leaves] leaves - [number of leaves]]
    - validate tree
    - if valid, return tree
    - else, ?
- Example:
    - x.length() = 3
    - number of leaves = 4
    - number of actual leaves = 8
    - number of coefficients = 1
    - number of operands = 3

    ```bash
                    +
            /                  \
            *                    +
        /      \           /        \
      u        1          1          u
    /   \    /   \      /   \       /  \
    nul  *  nul   *    nul    *     nul *
    ```
### EA approach
- Individual is rapresented as a tree and a fitness
    - fitness is a tuple of 2 values: (-mse, right_sign_100)
        - right_sign_100 is the percentage of correct sign predictions
        - mse is the mean squared error
- Classic Genetic Programming
    - Key elements 
    - Representation: tree structures
    - Recombination: exchange of subtrees
    - Mutation: random change in trees
        - subtree mutation -> replace  entire subtree
        - point  mutation -> change single node
        - permutation -> exchange node right with left
        - hoist -> take subtree and make it root
        - expansion -> take random leaf and replace it with a new subtree
        - collapse -> take a subtree and replace it with leaf
    - Population model: generational
    - Parent selection: fitness proportional
    - Survivor selection: deterministic

##next

    - finish implementing mutations
    - find way to reduce tree dimension
    - add weights to opertors
    - add check if fitness doesn't improve , stop early
    - 

##problems
    - overflow
    - initial tree with 4xnodes has invalid values
    - 


## Individual definition

In [65]:
@dataclass
class Individual:
    genome: Tree
    fitness: tuple

## Fitness evaluation

In [66]:
def fitness(sol: Tree):
    y_computed = sol.root.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS)
    right_sign_100 = 100 * np.sum(np.sign(y_computed) == np.sign(y)) / len(y)
    return  -mse(y_computed, y) * (np.exp((sol.get_max_depth() / 10))), right_sign_100
 
def sort_individuals(population, mse_weight=0.6):
    # Normalize both components to [0,1] scale across the population
    mse_scores = np.array([ind.fitness[0] for ind in population])
    sign_scores = np.array([ind.fitness[1] for ind in population])
    
    # can't sort if all mse's are equal
    diff_max_min = (mse_scores.max() - mse_scores.min())
    if diff_max_min == 0 :
        diff_max_min = 1
    # Min-max normalization
    mse_norm = (mse_scores - mse_scores.min()) / diff_max_min 
    sign_norm = sign_scores / 100  # Already in [0,100] range
    
    # Combine scores with weighted sum
    combined_scores = mse_weight * mse_norm + (1 - mse_weight) * sign_norm
    
    # sorted_population = [x for _, x in sorted(zip(combined_scores, population), 
    #                                         key=lambda pair: pair[0], 
    #                                         reverse=True)]

    # Sort population based on combined scores (descending order)
    sorted_pairs = sorted(zip(combined_scores, population), key=lambda pair: pair[0], reverse=True)
    sorted_scores, sorted_population = zip(*sorted_pairs)
    
    return list(sorted_population), list(sorted_scores)

def compute_score(individual, population, mse_weight=0.6):
     # Normalize both components to [0,1] scale across the population
    mse_scores = np.array([ind.fitness[0] for ind in population])
    sign_scores = np.array([ind.fitness[1] for ind in population])
    
    # Min-max normalization
    mse_norm = (individual.fitness[0] - mse_scores.min()) / (mse_scores.max() - mse_scores.min())
    sign_norm = individual.fitness[1] / 100  # Already in [0,100] range
    
    # Combine scores with weighted sum
    return mse_weight * mse_norm + (1 - mse_weight) * sign_norm


## Initial solution generation

In [ ]:
def generate_initial_solution(input_variables=list(VARIABLES_MAP.keys()), seed=None):
    if seed:
        random.seed(seed)
        np.random.seed(seed)
    variables = input_variables[:]
    n_variables = len(variables)
    if n_variables != 0:
        n_leaves = int(2 ** np.ceil(np.log2(n_variables)))
        n_actual_leaves = n_leaves * random.choice([2, 4])
        # n_actual_leaves = n_leaves * 2
        binary_operators = list(BINARY_OPERATORS.keys())[:]
        unary_operators = list(UNARY_OPERATORS.keys())[:]
        max_depth = np.log2(n_actual_leaves)
    else:
        KeyError("not enough vairbales in general_initial_solution")

    while True:
        root = random_initial_tree(0, max_depth, variables, binary_operators, unary_operators)
        try:
            #root.print_tree_from_node()
            if root.validate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS):
                root.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS)
                tree = Tree(root, max_depth)
                return tree
            else:
                print("not valid")
        except:
            pass
        variables = input_variables[:]
            
""" 
def wrapper(x):
    tree = generate_initial_solution(x)
    return evaluate_tree(tree.root)
"""

## Crossover and mutations

In [68]:
def parent_selection(population: list[Individual], scores, worst_score):
    # windowing    
    scores_prime = [(s-worst_score) for s in scores]
    # print(len(population), len(scores))
    total = sum(scores_prime)
    # if all population doesn't have some score
    if total != 0:
        probabilities = [s/total for s in scores_prime]

        parents = random.choices(population, probabilities, k=2)
            
        return parents[0], parents[1]
    
    return population[0], population[1]

def xover(parent1, parent2)-> tuple[Tree, tuple]:
    # parents = random.choices(population, k=2)

    # reproduce
    c1 = copy.deepcopy(parent1.genome)
    c2 = copy.deepcopy(parent2.genome)

    success = swap_subtrees(c1, c2)

    c_fitness = fitness(c2)

    if not success:
        # mutation
        mutation(c2)

    return c2, c_fitness


def subtree_mutation(target_tree: Tree):
    # generate a random source tree to substitute into target_tree

    num_variables = np.random.randint(1, len(VARIABLES_MAP.keys())) if len(VARIABLES_MAP.keys()) > 1 else 1  # take a random number of variables
    variables = random.sample(list(VARIABLES_MAP.keys()), num_variables)

    source_tree_root = generate_initial_solution(variables).root

    return try_swap(source_tree_root, target_tree, True)

def point_mutation(target_tree: Tree):
    nodes = target_tree.get_nodes()
    # target_tree.print_tree(VARIABLES_MAP)

    while nodes:
        node = random.choice(nodes)
        unary_operators = [op for op in  list(UNARY_OPERATORS.keys()) if op != node.value]
        unary_weights = {op: i for op,i in compute_weights_sim(unary_operators).items()}
        binary_operators = [op for op in list(BINARY_OPERATORS.keys()) if op != node.value]
        # mutate unary operator with another one
        if (node.value in UNARY_OPERATORS.keys()):
            while unary_operators:
                tmp = node.value
                # print(f"substituing : {tmp}")
                node.value = np.random.choice(list(unary_weights.keys()), p=list(unary_weights.values()))
                # print(f"chosen {node.value}")
                if are_compatible(node.value, node.right.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS)) and validate_after_replacement(target_tree.root, node):
                    # target_tree.print_tree(VARIABLES_MAP)
                    # print("ok")
                    return True
                unary_operators.remove(node.value)
                unary_weights = {op: i for op,i in compute_weights_sim(unary_operators).items()}
                node.value = tmp

        # mutate binary operator with another one
        elif (node.value in BINARY_OPERATORS.keys()):
            while binary_operators:
                tmp = node.value
                node.value = random.choice(binary_operators)
                if are_compatible(node.value, node.right.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS), node.left.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS)) and validate_after_replacement(target_tree.root, node):
                    return True
                binary_operators.remove(node.value)
                node.value = tmp

        # mutate variable
        elif node.value in VARIABLES_MAP.keys():
            # change the coefficient
            tmp_c = node.coefficient
            tmp_v = node.value
            node.coefficient = compute_coefficient(node.value, VARIABLES_MAP, coefficient_ranges)
            node.value = random.choice(list(VARIABLES_MAP.keys()))
            parent = target_tree.get_parent_from_root(node)
            if (parent.value in unary_operators and are_compatible(parent.value, parent.right.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS)) and validate_after_replacement(target_tree.root, parent)) \
            or (parent.value in binary_operators and are_compatible(parent.value, parent.right.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS), parent.left.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS)) and validate_after_replacement(target_tree.root, parent)):
                return True
            node.coefficient = tmp_c
            node.value = tmp_v
            
        # mutate constant value
        else :
            # change the constant
            tmp = node.value
            # node.value = compute_coefficient() # TODO change to compute constant
            parent = target_tree.get_parent_from_root(node)
            node.value = generate_constant(parent.value, UNARY_OPERATORS, y)
            if (parent.value in unary_operators and are_compatible(parent.value, parent.right.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS)) and validate_after_replacement(target_tree.root, parent)) \
            or (parent.value in binary_operators and are_compatible(parent.value, parent.right.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS), parent.left.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS)) and validate_after_replacement(target_tree.root, parent)):
                return True
            node.value = tmp
        
        nodes.remove(node)
    return False
    
def permutation_mutation(target_tree: Tree):
    available_nodes = [node for node in target_tree.get_non_leaves_nodes() if node.value in BINARY_OPERATORS.keys()]
    binary_operators = list(BINARY_OPERATORS.keys())
    unary_operators = list(UNARY_OPERATORS.keys())
    
    while available_nodes:  
        target_node = random.choice(available_nodes)
        tmpr = target_node.right
        tmpl = target_node.left
        target_node.right = tmpl
        target_node.left = tmpr
        if are_compatible(target_node.value, target_node.right.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS), target_node.left.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS)) and validate_after_replacement(target_tree.root, target_node):
            return True
        target_node.right = tmpr
        target_node.left = tmpl
        available_nodes.remove(target_node)
    return False

def expansion_mutation(target_tree: Tree):
    leaves = target_tree.get_leaves_nodes()
    binary_operators = list(BINARY_OPERATORS.keys())
    unary_operators = list(UNARY_OPERATORS.keys())
    num_variables = np.random.randint(1, len(VARIABLES_MAP.keys())) if len(VARIABLES_MAP.keys()) > 1 else 1  # take a random number of variables
    variables = random.sample(list(VARIABLES_MAP.keys()), num_variables)

    while leaves:
        target_node = random.choice(leaves)
        source_tree_root = generate_initial_solution(variables).root
        parent = target_tree.get_parent_from_root(target_node)

        if (parent.value in unary_operators and are_compatible(parent.value, source_tree_root.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS))):
            tmp = parent.right
            parent.right = source_tree_root
            if validate_after_replacement(target_tree.root, parent):
                target_tree.update_depths_to_root(parent)
                return True
            parent.right = tmp

        elif (parent.value in binary_operators):
            choice = random.choice(["right", "left"])
            if choice == "right" and are_compatible(parent.value, source_tree_root.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS), parent.left.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS)):
                tmp = parent.right
                parent.right = source_tree_root
                if validate_after_replacement(target_tree.root, parent):
                    target_tree.update_depths_to_root(parent)
                    return True
                parent.right = tmp
            elif choice == "left" and are_compatible(parent.value, parent.right.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS), source_tree_root.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS)):
                tmp = parent.left
                parent.left = source_tree_root
                if validate_after_replacement(target_tree.root, parent):
                    target_tree.update_depths_to_root(parent)
                    return True
                parent.left = tmp
        leaves.remove(target_node)
    return False

# substtitue the root with a subtree
def hoist_mutation(target_tree: Tree):
    nodes = target_tree.get_non_leaves_nodes()
    root = target_tree.root

    if len(nodes) == 1:
        return False
    
    #remove root
    nodes.remove(root)

    node = random.choice(nodes)

    target_tree.root = node
    return True

def collapse_mutation(target_tree: Tree):
    available_nodes = target_tree.get_non_leaves_nodes()

    # Consider only nodes whose children are not leaves
    available_nodes = [node for node in available_nodes if node.right.right or node.right.left]
    
    unary_operators = list(UNARY_OPERATORS.keys())
    binary_operators = list(BINARY_OPERATORS.keys())
    
    max_attempts = 10
    while available_nodes:
        parent = random.choice(available_nodes)
        attempts = 0
        while attempts < max_attempts:
            if random.choice([0, 1]): # create a radnom leaf
                random_leaf = get_random_leaf()
            else:
                random_leaf = copy.deepcopy(random.choice(parent.get_leaves_nodes_from_node()))

            if (parent.value in unary_operators and are_compatible(parent.value, random_leaf.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS))):
                tmp = parent.right
                parent.right = random_leaf
                if validate_after_replacement(target_tree.root, parent):
                    target_tree.update_depths_to_root(parent)
                    return True
                parent.right = tmp

            elif (parent.value in binary_operators):
                choice = random.choice(["right", "left"])
                if choice == "right" and are_compatible(parent.value, random_leaf.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS), parent.left.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS)):
                    tmp = parent.right
                    parent.right = random_leaf
                    if validate_after_replacement(target_tree.root, parent):
                        target_tree.update_depths_to_root(parent)
                        return True
                    parent.right = tmp
                elif choice == "left" and are_compatible(parent.value, parent.right.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS), random_leaf.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS)):
                    tmp = parent.left
                    parent.left = random_leaf
                    if validate_after_replacement(target_tree.root, parent):
                        target_tree.update_depths_to_root(parent)
                        return True
                    parent.left = tmp
            attempts += 1
        available_nodes.remove(parent)
    return False


MUTATIONS = {
    "subtree": subtree_mutation,
    "point": point_mutation,
    "permutation": permutation_mutation,
   "hoist": hoist_mutation,
    "expansion": expansion_mutation,
    "collapse": collapse_mutation
}

MUTATIONS_WEIGHTS = {
    "subtree": 0.25 , #0.25
    "point": 0.3 , #0.3
    "permutation": 0.1, #0.2 
    "hoist": 0.1, #0.1
    "expansion": 0.15, #0.15
    "collapse": 0.1 #0.1
 }


def mutation(genome: Tree):
    available_mutations = list(MUTATIONS.keys())
    available_mutations_weights = {m:p for m,p in MUTATIONS_WEIGHTS.items()}

    w_mutations = list(available_mutations_weights.values())
    while available_mutations:
        mutation = np.random.choice(available_mutations, p=w_mutations)
        # print(mutation)
        if MUTATIONS[mutation](genome):
            return True
        # scale weights 
        to_remove = w_mutations[available_mutations.index(mutation)]
        w_mutations.remove(to_remove)

        # recompute weights
        w_mutations = [w / sum(w_mutations) for w in w_mutations]

        available_mutations.remove(mutation)
    return False

In [ ]:
node = TreeNode("+")
node2 = TreeNode(3)
parent = node
tmp = parent
parent = node2

print(tmp.value)



## Main

In [77]:
POPULATION_SIZE = PROBLEM_SIZE*20
OFFSPRING_SIZE =int(POPULATION_SIZE / 4 )
MAX_ITERATIONS = POPULATION_SIZE*100

In [ ]:
a = [1,2,3,4,5,6,7,8,9,10,11,12]
p = []
for i in range(3):
    start =int( i*len(a)/3)
    end =int( (i+1)*len(a)/ 3)
    p_i = a[start:end]
    p.append(p_i)
p

### steady

In [96]:
# n. torunament --> population / #tour --> ea
# n.tour winners --> take best
# generate initial population from best with mutations

def tournament(population):
    # generate a certain number of tournaments
    n_tour = 3
    winners = []
    # for each tournament find a winner for a portion of the population
    for i in range(n_tour):
        start =int( i*len(population)/3)
        end =int( (i+1)*len(population)/ 3)
        p_i = population[start : end]

        w_i = ea(p_i)
        winners.append(w_i)
    
    winners , scores = sort_individuals(winners)
    # get the best winner/s
    # check if there are pari merito
    best_score = np.max(scores)

    return [winners[i] for i in range(len(winners)) if scores[i]==best_score]

def ea(population, iterations=200):
    # print(population)
    scores = []
    offsprings = []
    worst_score = 0
    for ind in population:
        ind.fitness = fitness(ind.genome)
    population, scores = sort_individuals(population)

    for i in tqdm(range(iterations)):
    
        for _ in range(OFFSPRING_SIZE):
            parent1, parent2 = parent_selection(population, scores, worst_score)
            # crossover
            if random.random() < 0.65:
                child, c_fitness = xover(parent1, parent2)
        
                offsprings.append(Individual(child, c_fitness))

            # mutation
            else :
                # take a random individual and mutate it
                ind = copy.deepcopy(random.choice([parent1,parent2]))

                mutation(ind.genome)
                # recompute fitness
                ind.fitness = fitness(ind.genome)
                offsprings.append(ind)
        population.extend(offsprings)
        offsprings = []
        
        # sort population according to score based on fitness         
        population, scores = sort_individuals(population)

        # remove worst individual
        population = population[:POPULATION_SIZE]
        scores = scores[:POPULATION_SIZE]

    print(f"winner's mse {mse(population[0].genome.root.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS), y)}")
    
    return population[0]

In [98]:
state = random.getstate()
population = [Individual(generate_initial_solution(seed=(i+2) * 10), 0) for i in range(POPULATION_SIZE)]
offsprings = []
random.setstate(state)

winners = tournament(population)
winners

[Individual(genome=<tree.Tree object at 0x0000028B30212B10>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B2F0C5910>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B2F0E61B0>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B2F0E6C30>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B2F039130>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B2EBF01A0>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B2FF87230>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B2FF84E90>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B2FF87530>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B2FF842C0>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B2FF862D0>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B2F03D7F0>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B2F0E75C0>, fitness=0)]
[Individual(genome=<tree.Tree object at 0x0000028B30212B10>, fi

  0%|          | 0/200 [00:00<?, ?it/s]

winner's mse 1.8146987521813582
[Individual(genome=<tree.Tree object at 0x0000028B2FF85700>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B2FE6E330>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B2FE6DF40>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B2FA3E5D0>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B302170B0>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B2FA3D670>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B30216F00>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B30215F10>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B30217C50>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B3021F1D0>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B2F03D970>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B3021DA90>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B2F0C4EC0>, fitness=0)]
[Individual(genome=<tree.Tree o

  0%|          | 0/200 [00:00<?, ?it/s]

winner's mse 28.86799908470968
[Individual(genome=<tree.Tree object at 0x0000028B3021DB80>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B2F0E7470>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B30226330>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B30214E60>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B2EBB4770>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B30215790>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B2F0E5EB0>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B30225460>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B30224920>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B30224DD0>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B30227800>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B302278F0>, fitness=0), Individual(genome=<tree.Tree object at 0x0000028B30227890>, fitness=0), Individual(genome=<tree.Tree obj

  0%|          | 0/200 [00:00<?, ?it/s]

winner's mse 31.510401197912408


[Individual(genome=<tree.Tree object at 0x0000028B2F02C6B0>, fitness=(np.float64(-2.991932432634386), np.float64(98.2)))]

In [ ]:
state = random.getstate()
population = [Individual(generate_initial_solution(seed=(i+2) * 10), 0) for i in range(POPULATION_SIZE)]
offsprings = []
random.setstate(state)

# torunament -> use initial population
winner = tournament(population)
#generate population from winner

# set the fitness
for ind in population:
    ind.fitness = fitness(ind.genome)
population, scores = sort_individuals(population)
population[0].genome.draw_tree()

# print(population)

best = None
best_score = -1
worst_score = 0

prev_score = 0 #score to check every 10% of iterations
scores = [compute_score(population[i], population) for i in range(POPULATION_SIZE)]

for i in tqdm(range(MAX_ITERATIONS)):
    
    for _ in range(OFFSPRING_SIZE):
        parent1, parent2 = parent_selection(population, scores, worst_score)
        # crossover
        if random.random() < 0.65:
            child, c_fitness = xover(parent1, parent2)
    
            offsprings.append(Individual(child, c_fitness))

        # mutation
        else :
            # take a random individual and mutate it
            ind = copy.deepcopy(random.choice([parent1,parent2]))

            mutation(ind.genome)
            # recompute fitness
            ind.fitness = fitness(ind.genome)
            offsprings.append(ind)
    population.extend(offsprings)
    offsprings = []
    
    # sort population according to score based on fitness         
    population, scores = sort_individuals(population)
    # print(f"{i}: scores {scores}\n")

    if i % 100 == 0:
        print(f"iteration {i} best individual : {population[0].fitness}")
        # compute score 
        score = scores[0]
        if score > best_score:
            print(f"new best score : {score}, old score : {best_score}")
            best = population[0]
            best_score = score

        # put again the best one in population
        else: 
            print("valhalla")
            population.append(best)
            population,scores = sort_individuals(population)
            # print(f"valhallla {i}: scores {scores}\n")

        worst_score = scores[OFFSPRING_SIZE-1] # if worst_score > scores[-1] else worst_score
   
   
    # remove worst individual
    population = population[:POPULATION_SIZE]
    scores = scores[:POPULATION_SIZE]
    
    # stop searching if scores doesn't improve anymore after 20% of max iterations
    if i % (MAX_ITERATIONS*0.25) == 0:
        if prev_score == scores[0]:
            print("exit")
            break
        else:
            prev_score = scores[0]

    

print("Best individual has formula:")
print(f"Best individual mse {mse(population[0].genome.root.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS), y)}")
population[0].genome.draw_tree()

## version 2

In [72]:
POPULATION_SIZE = PROBLEM_SIZE*20
OFFSPRING_SIZE = POPULATION_SIZE*3
MAX_ITERATIONS = 1000

In [ ]:
state = random.getstate()
initial_population = [Individual(generate_initial_solution(seed=(i+2) * 10), 0) for i in range(POPULATION_SIZE)]
offsprings = []
random.setstate(state)

# set the fitness
for ind in initial_population:
    ind.fitness = fitness(ind.genome)
initial_population, scores = sort_individuals(initial_population)
initial_population[0].genome.draw_tree()

# print(initial_population)

best = None
best_score = 0
worst_score = 0
scores = [compute_score(initial_population[i], initial_population) for i in range(POPULATION_SIZE)]

for i in tqdm(range(MAX_ITERATIONS)):
# for i in range(MAX_ITERATIONS):
    for _ in range(OFFSPRING_SIZE):
        parent1, parent2 = parent_selection(initial_population, scores, worst_score)
        # crossover
        if random.random() < 0.65:
            child, c_fitness = xover(parent1, parent2)
    
            offsprings.append(Individual(child, c_fitness))

        # mutation
        else :
            # take a random individual and mutate it
            ind = copy.deepcopy(random.choice([parent1,parent2]))

            mutation(ind.genome)
            # recompute fitness
            ind.fitness = fitness(ind.genome)
            offsprings.append(ind)

    
    # sort population according to score based on fitness         
    initial_population, scores = sort_individuals(offsprings)
    # print(f"{i}: scores {scores}\n")

    if i % 100 == 0:
        print(f"iteration {i} best individual : {initial_population[0].fitness}")
        # compute score 
        score = scores[0]
        if score > best_score:
            print(f"new best score : {score}, old score : {best_score}")
            best = initial_population[0]
            best_score = score

        # put again the best one in population
        else: 
            print("valhalla")
            initial_population.append(best)
            initial_population,scores = sort_individuals(initial_population)
            print(f"valhallla {i}: scores {scores}\n")

        worst_score = scores[OFFSPRING_SIZE-1] # if worst_score > scores[-1] else worst_score
   
   
    # remove worst individual
    initial_population = initial_population[:POPULATION_SIZE]
    scores = scores[:POPULATION_SIZE]
    # print(f"cut {i}: scores {scores}\n")
 
    # update the worst score
    # print(f"iteration {i} ")
    offsprings = []

print("Best individual has formula:")
print(f"Best individual mse {mse(initial_population[0].genome.root.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS), y)}")
initial_population[0].genome.draw_tree()